In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TORCH_HOME'] = "/disks/sdb/torch_home"

import logging
logging.getLogger().setLevel(logging.WARNING)

from awesome_glue.task import *

Loading faiss with AVX2 support.
I0304 10:24:25.523280 140083882231616 file_utils.py:38] PyTorch version 1.4.0 available.
I0304 10:24:26.868705 140083882231616 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I0304 10:24:27.076905 140083882231616 modeling_bert.py:244] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
[nltk_data] Downloading package wordnet to /home/zhouyi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/zhouyi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
data = load_data('AGNEWS', 'spacy')

cache for AGNEWS-spacy.data exists
*** load AGNEWS-spacy.data from cache cost 14.9 seconds


In [27]:
reader = SpacyTSVReader(sent1_col='sentence',
                 sent2_col=None,
                 label_col='label',
                 skip_label_indexing=True)
data = reader.read("datasets/glue_data/SST-2/dev.tsv")

In [28]:
vocab = Vocabulary.from_instances(data)

I0304 10:30:44.827859 140083882231616 vocabulary.py:438] Fitting token dictionary from dataset.


In [54]:
import re
re.sub("\t", "", "abc")

'abc'

In [82]:
import json
fin = open("/home/zhouyi/imdb/test.jsonl")
fout = open("datasets/imdb/test.tsv", 'w')
fout.write("id\tsentence\tlabel\n")
idx = 0
for ln in fin:
    idx += 1
#     print(idx)
    x = json.loads(ln)

    text = x['text']
    text = re.sub(r"[^A-Za-z0-9(),:;.!?\'\`\-\"]", " ", text)

    
    text = re.sub(r' +', ' ', text)
    
    fout.write(f"{x['id']}\t{text}\t{x['label']}\n")
fout.close()    